In [1]:
from vllm import LLM, SamplingParams

model = LLM("/home/xusheng_liang/cair/xtuner-main/1_8b_40k", download_dir="/gscratch/h2lab/akari/model_cache", dtype="half")
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=100, skip_special_tokens=False)



WARNING 07-30 22:15:34 _custom_ops.py:14] Failed to import from vllm._C with ImportError('/home/xusheng_liang/anaconda3/lib/python3.11/site-packages/vllm/_C.abi3.so: undefined symbol: _ZN3c104cuda14ExchangeDeviceEa')


RuntimeError: Failed to load the model config. If the model is a custom model not yet available in the HuggingFace transformers library, consider setting `trust_remote_code=True` in LLM or using the `--trust-remote-code` flag in the CLI.

In [ ]:
def format_prompt(input, paragraph=None):
  prompt = "### Instruction:\n{0}\n\n### Response:\n".format(input)
  if paragraph is not None:
    prompt += "[Retrieval]<paragraph>{0}</paragraph>".format(paragraph)
  return prompt

input = '''
Case Summary\nPatient Basic Information: Middle-aged male, XX years old. (We anonymize the age information in the sample data presented.)\nChief Complaint: Vomiting blood for 2 days after eating.\nMedical History: The patient experienced vomiting of coffee-colored gastric contents (approximately 100ml) accompanied by dizziness, palpitations, and weakness after consuming hard food 2 days ago. There was no abdominal distension, pain, melena, or bloody stool, nor any confusion. The patient was treated conservatively with acid-suppressing and hemostatic medications, after which symptoms of vomiting blood improved. The patient has a history of chronic Hepatitis B for three years, which has not been treated. \nPhysical Examination: Pale skin and mucous membranes, flat abdomen with no visible peristaltic waves and presence of abdominal breathing. No abdominal wall vein varicosity was observed. The abdomen was soft without fluid wave or shifting dullness, and no palpable masses. There was no significant tenderness or rebound tenderness, and the liver and spleen were not palpable below the ribs. Murphy's sign was negative. No evident kidney area tenderness or percussion pain, and no abnormal vascular pulsation in the abdomen. No significant tenderness at bilateral ureteral pressure points. Liver dullness was present, with the upper boundary at the right mid-clavicular line at the fifth intercostal space, with no shifting dullness. Bowel sounds were normal. \nAuxiliary Examination：\n（1）Imaging Examination：\nCT Scan (Plain + Contrast): 1. Ground glass nodule in the lower lobe of the right lung, suggest a follow-up CT in 3-6 months. 2. Linear opacities in the lower lobes of both lungs. 3. Liver cirrhosis, splenomegaly, varices at the lower end of the esophagus and the gastric fundus, and varices in front of the spleen. 4. Possible subcapsular hemangioma in liver segment S7, further examination with MRI suggested. 5. Multiple small cysts in the right lobe of the liver. 6. Fluid accumulation in the gallbladder fossa. 7. No apparent abnormalities in the lower abdominal CT scan.\nMRI (Plain + Contrast)：1. Liver cirrhosis, fibrosis; enlarged spleen; portal hypertension. 2. Small cyst in liver segment S5. 3. Minor fluid accumulation in the gallbladder fossa.\nEsophagogastroduodenoscopy: 1. Esophageal varices rupture treated with banding and tissue glue sclerotherapy. 2. Esophageal drug injection via gastroscopy. 3. Endoscopic hemostasis. 4. Portal hypertensive gastropathy.\n（2）Laboratory Examination：\nRoutine Blood Test: 1. Red Blood Cells (RBC) 3.0*10^12/L ↓; 2. Hemoglobin (HGB) 97g/L ↓; 3. Hematocrit (HCT) 27.9% ↓; 4. Platelet Count (Impedance Method) (PLT-I) 47*10^9/L ↓; 5. Mean Platelet Volume (MPV) 13.2fL ↑; 6. Plateletcrit (PCT) 0.06% ↓.\nBlood Biochemistry Test: 1. Aspartate Aminotransferase (AST) 60U/L ↑; 2. Total Protein (TP) 61.6g/L ↓; 3. Albumin (ALB) 31.7g/L ↓; 4. Albumin/Globulin Ratio (A/G) 1.11.2-2.4 ↓; 5. Total Bilirubin (TBIL) 41.5μmol/L ↑; 6. Direct Bilirubin (DBIL) 10.0μmol/L ↑; 7. Indirect Bilirubin (IBIL) 31.5μmol/L ↑; 8. Prealbumin (PA) 93.5mg/L ↓; 9. Calcium (Ca) 2.10mmol/L ↓; 10. Sodium (Na) 136mmol/L ↓; 11. Osmotic Pressure (OSM) 272mOsm/kg ↓.\nCoagulation Function Test: 1. Prothrombin Time# (PT#) 20.8S ↑; 2. Thrombin Time# (TT#) 19.5S ↑; 3. Fibrinogen# (Fg#) 1.1g/L ↓; 4. Percentage Activity (PT%) 43% ↓; 5. International Normalized Ratio (PT.INR) 1.810.85-1.25 ↑.\nTumor Marker Test: 1. Alpha-Fetoprotein (AFP) 307.2ng/mL ↑; 2. Carbohydrate Antigen 19-9 (CA19-9) 69.9U/mL ↑.\n(3) Pathological Examination: None at the moment.
'''
queries = f'''Can you tell me which department I should go to if my medical my leg broken.Depatments are as follow:"Medical Intensive Care Unit","Surgical Intensive Care Unit","Cardiac Vascular Intensive Care Unit","Coronary Care Unit","Neuro Intermediate","Trauma SICU","Neuro Stepdown","Neuro Surgical Intensive Care Unit","Surgical Intensive Care Unit (MICU/SICU)"'''

# for a query that doesn't require retrieval
preds = model.generate(queries, sampling_params)
for pred in preds:
  print("Model prediction: {0}".format(pred.outputs[0].text))

In [15]:
import requests
import json
def cares_copilot(query):
    url = 'http://175.45.13.117:53999/v1/chat/completions'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    data = {
        "model": "internlm2",
        "messages": [
            {"role": "user", "content": f'''You are an ICU doctor. Please make a diagnosis for the patient based on the patient's medical records provided. The diagnosis must belong to the following list of diseases. [“Symptoms, signs and abnormal clinical and laboratory findings”,
“Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism”,
“Diseases of the musculoskeletal system and connective tissue”,
“Diseases of the skin and subcutaneous tissue”,
“Mental and behavioural disorders”,
“Diseases of the genitourinary system”,
“Congenital malformations, deformations and chromosomal abnormalities”,
“Certain infectious and parasitic diseases”,
“Pregnancy, childbirth and the puerperium”,
“Endocrine, nutritional and metabolic Diseases”,
“Diseases of the circulatory system”,
“Diseases of the digestive system”,
“Diseases of the respiratory system”,
“Injury, poisoning and certain other consequences of external causes”,
“Certain infectious and parasitic Diseases”,
“Diseases of the nervous system and sense organs”,
“Neoplasms”],Please output no more than three types of diseases.
\n\nThe mdecial reocrd is as follows: {query}'''}
        ],
        "temperature": 0,
        "top_p": 0.8,
        "n": 1,
        "max_tokens": None,
        "stop": None,
        "stream": False,
        "presence_penalty": 0,
        "frequency_penalty": 0,
        "user": "string",
        "repetition_penalty": 1.002,
        "session_id": -1,
        "ignore_eos": False
    }
    response = requests.post(url, headers=headers, json=data)
    response_json = response.json()
    return response_json['choices'][0]['message']['content']

In [16]:
with open("/home/xusheng_liang/cair/MetaGPT-main/examples/clinic/data/raft/test_721.json","r") as file:
    data = json.load(file)
for item in data:
    query= item["query"]
    response = cares_copilot(query)
    item["answer"] = response

with open("/home/xusheng_liang/cair/MetaGPT-main/examples/clinic/data/InternLM2/new_results_92k_721_withpred.json", 'w', encoding='utf-8') as output_file:
    json.dump(data, output_file, ensure_ascii=False, indent=4)

KeyboardInterrupt: 

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("/home/xusheng_liang/cair/xtuner-main/RAG/7b_80k", trust_remote_code=True)
# Set `torch_dtype=torch.float16` to load model in float16, otherwise it will be loaded as float32 and might cause OOM Error.
model = AutoModelForCausalLM.from_pretrained("/home/xusheng_liang/cair/xtuner-main/RAG/7b_80k", trust_remote_code=True, device_map="auto",torch_dtype=torch.float16)
# (Optional) If on low resource devices, you can load model in 4-bit or 8-bit to further save GPU memory via bitsandbytes.
  # InternLM 7B in 4bit will cost nearly 8GB GPU memory.
  # pip install -U bitsandbytes
  # 8-bit: model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", trust_remote_code=True, load_in_8bit=True)
  # 4-bit: model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", trust_remote_code=True, load_in_4bit=True)
model = model.eval()
# response, history = model.chat(tokenizer, "hello", history=[])
# print(response)
# # Output: Hello? How can I help you today?
# response, history = model.chat(tokenizer, "please provide three suggestions about time management", history=history)
# print(response)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
# history = ['''Please select a disease from the following list based on the patient's case report:Certain infectious and parasitic diseases,
# Neoplasms,
# Endocrine, nutritional and metabolic Diseases,
# Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism,
# Mental and behavioural disorders,
# Diseases of the nervous system and sense organs,
# Diseases of the circulatory system,
# Diseases of the respiratory system,
# Diseases of the digestive system,
# Diseases of the genitourinary system,
# Pregnancy, childbirth and the puerperium,
# Diseases of the skin and subcutaneous tissue,
# Diseases of the musculoskeletal system and connective tissue,
# Congenital malformations, deformations and chromosomal abnormalities,
# Certain conditions originating in the perinatal period,
# Symptoms, signs and abnormal clinical and laboratory findings,
# Injury, poisoning and certain other consequences of external causes''']
import json
from tqdm import tqdm 
with open("/home/xusheng_liang/cair/MetaGPT-main/examples/clinic/data/ex/test_721_correlation.json","r") as file:
    data = json.load(file)
num = 0
for item in data:

    # response = cares_copilot(query)
    # item["answer"] = response



    idea = item["query"]
    query = f'''{idea}\nPlease make a diagnosis based on the patient's case from the following list based on the patient's case report:"Certain infectious and parasitic diseases",
    "Neoplasms",
    "Endocrine, nutritional and metabolic Diseases",
    "Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism",
    "Mental and behavioural disorders",
    "Diseases of the nervous system and sense organs",
    "Diseases of the circulatory system",
    "Diseases of the respiratory system",
    "Diseases of the digestive system",
    "Diseases of the genitourinary system",
    "Pregnancy, childbirth and the puerperium",
    "Diseases of the skin and subcutaneous tissue",
    "Diseases of the musculoskeletal system and connective tissue",
    "Congenital malformations, deformations and chromosomal abnormalities",
    "Certain conditions originating in the perinatal period",
    "Symptoms, signs and abnormal clinical and laboratory findings",
    "Injury, poisoning and certain other consequences of external causes" '''

    response, history = model.chat(tokenizer, query, history=[])
    item["internlm2_1_8b_directanswer"] = response
    print(response)
    num+=1
    print(num)
with open("/home/xusheng_liang/cair/MetaGPT-main/examples/clinic/data/ex/test_721_internlm2_1_8b.json", 'w', encoding='utf-8') as output_file:
    json.dump(data, output_file, ensure_ascii=False, indent=4)

Based on the patient's case report, it appears that the patient is suffering from a congenital abnormality involving the right wrist. Specifically, the patient has multiple congenital abnormalities including esophageal atresia, status post tracheoesophageal fistula repair, pyloric stenosis, status post appendectomy, and absence of the right thumb. The patient also has a history of small bowel obstruction and multiple right upper extremity surgeries. The patient's findings suggest that the right wrist is remodeled and attenuated distally, with the right radius appearing healed and with an intact screw fixating the third metacarpal to the distal radius. There is an intact screw along the distal margin of the radius and a second screw is noted. A third screw is noted in the distal radius.

Based on this information, a diagnosis of the patient's congenital wrist abnormality could be made. However, further evaluation and consultation with a medical professional would be necessary to confirm

In [13]:
response

'Based on the provided patient case report, it is difficult to make a specific diagnosis as the report is quite vague and lacks necessary details. However, some potential diagnoses that could be considered are:\n\n1. Malignancies: The report mentions "extremely limited exam due to soft tissue attenuation, motion, and bolustiming." Soft tissue involvement could suggest a soft tissue malignancy, such as a lipoma or a sarcoma. However, more information is needed to make a definitive diagnosis.\n\n2. Endocrine, nutritional, and metabolic diseases: The report mentions "no main, right main, or left main pulmonary artery pulmonary embolism." This could indicate a pulmonary embolism related to an endocrine, nutritional, or metabolic disorder.\n\n3. Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism: The report mentions "no central obstructed bronchus" and "no suspicious osseous abnormality." These findings could suggest a condition affecting the

In [ ]:
import json
with open("/home/xusheng_liang/cair/MetaGPT-main/examples/clinic/data/ex/test_721_correlation.json","r") as file:
    data = json.load(file)
for item in data:
    query= item["query"]
    response, history = model.chat(tokenizer, "hello", history=[])
    item["internlm2_1_8b_answer"] = response

with open("# response, history = model.chat(tokenizer, "hello", history=[])", 'w', encoding='utf-8') as output_file:
    json.dump(data, output_file, ensure_ascii=False, indent=4)


In [1]:
from openai import OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://172.21.44.9:11390/v1"

client = OpenAI(
# defaults to os.environ.get("OPENAI_API_KEY")
api_key=openai_api_key,
base_url=openai_api_base,
)

models = client.models.list()
model = models.data[0].id


# def get_result(prompt):
#     response = client.chat.completions.create(
#     messages=[
#     {"role": "system", "content": "You are a helpful assistant"},
#     {"role": "user", "content": prompt},
#     ],
#     stream=False,
#     temperature=1.25
#     )
#     return    

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("/home/xusheng_liang/cair/xtuner-main/RAG/1_8b_92k", trust_remote_code=True)
# Set `torch_dtype=torch.float16` to load model in float16, otherwise it will be loaded as float32 and might cause OOM Error.
model = AutoModelForCausalLM.from_pretrained("/home/xusheng_liang/cair/xtuner-main/RAG/1_8b_92k", trust_remote_code=True, device_map="auto",torch_dtype=torch.float16)
# (Optional) If on low resource devices, you can load model in 4-bit or 8-bit to further save GPU memory via bitsandbytes.
  # InternLM 7B in 4bit will cost nearly 8GB GPU memory.
  # pip install -U bitsandbytes
  # 8-bit: model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", trust_remote_code=True, load_in_8bit=True)
  # 4-bit: model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", trust_remote_code=True, load_in_4bit=True)
model = model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import json
import re
with open('/home/xusheng_liang/cair/ex/test_corr&hints.json', 'r') as file:
    data = json.load(file)
num = 0 

for item in data:
    query = item["query"]
    Instruction = item["Instruction"]
    # diagnosis = item["top3_disease"]
    # cleaned_text = re.sub(r"'Past Medical History':.*?,\s*'radio_text'", "'Past Medical History':'', 'radio_text'", query, flags=re.DOTALL)
    radio = item["radio_text"]
    if item["correlation"] == "Yes":
        prompt = "Medical record:" + query  + Instruction + "Please give a diagnosis based on the patient's personal information, past medical history, and radiology report.Pay attention to the instructor's tips, that's crucial"
        # idea = item["query"]
        response, history = model.chat(tokenizer, prompt, history=[])
        item["Intern_answer"] = response
    else:
        prompt = "Medical record:" + radio  + Instruction + "Please give a diagnosis based on the patient's personal information and radiology report.Pay attention to the instructor's tips, that's crucial"
        # idea = item["query"]
        response, history = model.chat(tokenizer, prompt, history = [])
        item["Intern_answer"] = response

    # print(response)
    num+=1
    print(num)
    with open('/home/xusheng_liang/cair/ex/test_721_oursbest1_8b.json', 'w', encoding='utf-8') as output_file:
        json.dump(data, output_file, ensure_ascii=False, indent=4)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [8]:
import json
import transformers
import torch
import json
import random
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
model_id = "/home/xusheng_liang/cair/xtuner-main/RAG/LLaMA3_8B_Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    # device_map=0,
)

def disease_test(query):
    messages = [
    {"role": "system", "content": '''You are a clinic expert and need to make a diagnose for possible diseases from the candidate disease list based on the patient's medical record information. The candidate disease list is as follows:["Certain infectious and parasitic diseases","Neoplasms","Endocrine, nutritional and metabolic Diseases","Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism","Mental and behavioural disorders","Diseases of the nervous system and sense organs","Diseases of the circulatory system","Diseases of the respiratory system","Diseases of the digestive system","Diseases of the genitourinary system","Pregnancy, childbirth and the puerperium","Diseases of the skin and subcutaneous tissue","Diseases of the musculoskeletal system and connective tissue","Congenital malformations, deformations and chromosomal abnormalities","Certain conditions originating in the perinatal period","Symptoms, signs and abnormal clinical and laboratory findings","Injury, poisoning and certain other consequences of external causes"]'''},
    {"role": "user", "content": query},
]

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=4096,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    return outputs[0]["generated_text"][-1]["content"]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.61it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [9]:
disease_test("666")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


"As a clinic expert, I'll make a diagnosis based on the patient's medical record information. Since you didn't provide any information, I'll assume that the patient has presented with symptoms and I'll make a diagnosis based on the patient's symptoms.\n\nPlease provide me with the patient's medical record information, including their symptoms, medical history, and any relevant test results."

In [1]:
def extract_department(text):
    # 查找 "Diagnostic Department:" 的位置
    start = text.find("Diagnostic Department:")
    if start == -1:
        return None  # 如果没有找到这个标记，返回 None
    
    # 从标记之后开始查找换行符
    end = text.find("\n", start)
    if end == -1:
        end = len(text)  # 如果没有找到换行符，使用整个字符串的长度
    
    # 提取部门名称并去除前后的空白字符
    department = text[start + len("Diagnostic Department:"):end].strip()
    
    return department

# 测试函数
text = "Diagnostic Department: Medical Intensive Care Unit (MICU)\n\nReasons:\n1. The patient's medical history includes cardiac issues such as pericarditis, hypertension, and dyslipidemia, which indicates a potential need for close monitoring and management in a cardiac care unit.\n2. The radiology report mentions mildly enlarged heart size and mild unfolding of the thoracic aorta, which further supports the need for cardiac monitoring and intervention if necessary.\n3. The patient also has a history of rheumatoid arthritis, which may require additional monitoring and management in an intensive care setting.\n4. The presence of mild bibasilar atelectasis suggests possible respiratory complications that may require close monitoring and intervention in an intensive care unit.\n5. The patient's single marital status and past medical history involving multiple surgeries and medical procedures indicate a higher risk profile, making an intensive care unit the most appropriate setting for comprehensive care and monitoring.\n\nGiven these factors, the Medical Intensive Care Unit (MICU) is the most suitable diagnostic department for this patient. The MICU provides specialized care for patients with complex medical conditions, including cardiac and respiratory issues, and is equipped to handle patients with a higher risk profile."

department = extract_department(text)
print(department)

Medical Intensive Care Unit (MICU)


In [8]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    cosine_sim = cosine_similarity(vectorizer)
    return cosine_sim[0][1]

def process_idea(idea, threshold=0.9):
    # 提取 Past Medical History 和 Radiology Report
    pmh_match = re.search(r'Past Medical History:(.*?)(?=\n\s*\S+:|\Z)', idea, re.DOTALL)
    rr_match = re.search(r'Radiology Report:(.*?)(?=\n\s*\S+:|\Z)', idea, re.DOTALL)
    
    if pmh_match and rr_match:
        pmh = pmh_match.group(1).strip()
        rr = rr_match.group(1).strip()
        
        # 计算相关性
        similarity = calculate_similarity(pmh, rr)
        print(similarity)
        if similarity < threshold:
            # 如果相关性小于阈值，移除 Past Medical History 及其之后的内容
            new_idea = re.sub(r'Past Medical History:.*', '', idea, flags=re.DOTALL).strip()
            return new_idea
    
    # 如果相关性大于等于阈值或无法提取所需信息，返回原始 idea
    return idea

# 测试函数
idea = '''
Medical Reocrd:
    Language:English
    Gender:Male
    Marital status:Single
    Race:White
    Radiology Report:EXAMINATION:  Chest radiographINDICATION:  Dyspnea.TECHNIQUE:  AP and lateral views of the chest.COMPARISON:  ___FINDINGS: Heart size is mildly enlarged.  There is mild unfolding of the thoracic aorta.Cardiomediastinal silhouette and hilar contours are otherwise unremarkable. There is mild bibasilar atelectasis.  Lungs are otherwise clear.  Pleuralsurfaces are clear without effusion or pneumothorax.  Focus of air seen underthe right hemidiaphragm, likely represents colonic interposition.IMPRESSION: No acute cardiopulmonary abnormality.
    Past Medical History:Cardiac History  -Pericarditis, as above.-Hypertension.-Dyslipidemia.   Other PMH  -Rheumatoid arthritis. -Remote traumatic DVT.-Cholecystectomy.-Appendectomy.-Tonsillectomy.-Left wrist reconstruction.-Right rotator cuff reconstruction. 
'''

result = process_idea(idea)
print(result)

0.36245040183784216
Medical Reocrd:
    Language:English
    Gender:Male
    Marital status:Single
    Race:White
    Radiology Report:EXAMINATION:  Chest radiographINDICATION:  Dyspnea.TECHNIQUE:  AP and lateral views of the chest.COMPARISON:  ___FINDINGS: Heart size is mildly enlarged.  There is mild unfolding of the thoracic aorta.Cardiomediastinal silhouette and hilar contours are otherwise unremarkable. There is mild bibasilar atelectasis.  Lungs are otherwise clear.  Pleuralsurfaces are clear without effusion or pneumothorax.  Focus of air seen underthe right hemidiaphragm, likely represents colonic interposition.IMPRESSION: No acute cardiopulmonary abnormality.


In [5]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    cosine_sim = cosine_similarity(vectorizer)
    return cosine_sim[0][1]

def process_idea(record, threshold=0.3):
    # 提取 Past Medical History 和 Radiology Report
    pmh_match = re.search(r'Past Medical History:(.*?)(?=\n\s*\S+:|\Z)', record, re.DOTALL)
    rr_match = re.search(r'Radiology Report:(.*?)(?=\n\s*\S+:|\Z)', record, re.DOTALL)
    
    if pmh_match and rr_match:
        pmh = pmh_match.group(0).strip()  # 包含 "Past Medical History:" 标签
        rr = rr_match.group(1).strip()
        
        # 计算相关性
        similarity = calculate_similarity(pmh, rr)
        print(similarity)
        if similarity < threshold:
            # 如果相关性小于阈值，只删除 Past Medical History 部分
            new_record = re.sub(r'Past Medical History:.*?(?=\n\s*\S+:|\Z)', '', record, flags=re.DOTALL)
            return new_record.strip()
    
    # 如果相关性大于等于阈值或无法提取所需信息，返回原始 idea
    return record

# 测试函数
record = '''
Medical Reocrd:
    Language:English
    Gender:Male
    Marital status:Single
    Race:White
    Radiology Report:EXAMINATION:  Chest radiographINDICATION:  Dyspnea.TECHNIQUE:  AP and lateral views of the chest.COMPARISON:  ___FINDINGS: Heart size is mildly enlarged.  There is mild unfolding of the thoracic aorta.Cardiomediastinal silhouette and hilar contours are otherwise unremarkable. There is mild bibasilar atelectasis.  Lungs are otherwise clear.  Pleuralsurfaces are clear without effusion or pneumothorax.  Focus of air seen underthe right hemidiaphragm, likely represents colonic interposition.IMPRESSION: No acute cardiopulmonary abnormality.
    Past Medical History:Cardiac History  -Pericarditis, as above.-Hypertension.-Dyslipidemia.   Other PMH  -Rheumatoid arthritis. -Remote traumatic DVT.-Cholecystectomy.-Appendectomy.-Tonsillectomy.-Left wrist reconstruction.-Right rotator cuff reconstruction. 
'''

result = process_idea(record)
print(result)

0.3809777821323634

Medical Reocrd:
    Language:English
    Gender:Male
    Marital status:Single
    Race:White
    Radiology Report:EXAMINATION:  Chest radiographINDICATION:  Dyspnea.TECHNIQUE:  AP and lateral views of the chest.COMPARISON:  ___FINDINGS: Heart size is mildly enlarged.  There is mild unfolding of the thoracic aorta.Cardiomediastinal silhouette and hilar contours are otherwise unremarkable. There is mild bibasilar atelectasis.  Lungs are otherwise clear.  Pleuralsurfaces are clear without effusion or pneumothorax.  Focus of air seen underthe right hemidiaphragm, likely represents colonic interposition.IMPRESSION: No acute cardiopulmonary abnormality.
    Past Medical History:Cardiac History  -Pericarditis, as above.-Hypertension.-Dyslipidemia.   Other PMH  -Rheumatoid arthritis. -Remote traumatic DVT.-Cholecystectomy.-Appendectomy.-Tonsillectomy.-Left wrist reconstruction.-Right rotator cuff reconstruction. 



In [6]:
aa = [{'role': 'Doctor_Intern', 'content': "```Diagnostic results: \n\nBased on the patient's medical record and radiology report, I can deduce the following diagnosis:\n\nThe patient is a male, single, and white individual who presented with dyspnea. The radiology report reveals mild bibasilar atelectasis, mild unfolding of the thoracic aorta, and a focus of air seen under the right hemidiaphragm, likely representing colonic interposition.\n\nGiven these findings, I would suspect that the patient may be suffering from a respiratory disease, possibly related to the mild bibasilar atelectasis. The mild unfolding of the thoracic aorta could be indicative of a cardiovascular condition, but it is not severe enough to be considered acute.\n\nBased on the patient's symptoms and radiological findings, I would recommend further evaluation and testing to rule out any underlying respiratory or cardiovascular conditions. Some possible diagnoses to consider include:\n\n* Chronic obstructive pulmonary disease (COPD)\n* Pneumonia\n* Pleurisy\n* Aortic aneurysm or dissection\n* Colonic interposition (which may be related to a previous surgical procedure)\n\nIt is important to note that the patient's medical history and radiology report do not suggest any acute cardiopulmonary abnormality, but further evaluation and testing may be necessary to determine the underlying cause of the patient's symptoms.\n```\n\n```Current_service results: \n\nBased on the patient's medical record, it appears that the patient is experiencing dyspnea (shortness of breath) and has been diagnosed with mild bibasilar atelectasis and a focus of air under the right hemidiaphragm, likely representing colonic interposition. Given these findings, I would recommend the following services:\n\n1. SURG (Surgery) - Given the presence of a focus of air under the right hemidiaphragm, it is likely that the patient may require surgical intervention to address this issue.\n2. VSURG (Vascular Surgery) - Given the mild unfolding of the thoracic aorta, it is possible that the patient may require vascular surgery to address this issue.\n\nBased on the patient's current healthcare needs, I would recommend considering both SURG and VSURG services.\n```"}, {'role': 'Doctor_Chief', 'content': "Is it true?:\n\nYes, the patient's medical record does indicate dyspnea, mild bibasilar atelectasis, mild unfolding of the thoracic aorta, and a focus of air under the right hemidiaphragm, likely representing colonic interposition. However, the diagnosis and treatment plan proposed by Doctor_Intern could be improved.\n\nGuidance:\n\nWhile the proposed diagnoses are reasonable, it's important to consider the patient's medical history and the absence of acute cardiopulmonary abnormalities. The mild bibasilar atelectasis and unfolding of the thoracic aorta may not be indicative of severe respiratory or cardiovascular conditions, respectively. \n\nInstead, focus on addressing the patient's dyspnea and the potential cause of the focus of air under the right hemidiaphragm. Further evaluation, such as a CT scan of the chest and abdomen, may be necessary to determine the exact cause of the patient's symptoms. \n\nIn terms of treatment, surgical intervention may not be necessary at this stage. Conservative management, including oxygen therapy, bronchodilators, and antibiotics if indicated, should be considered first. If the patient's symptoms persist or worsen, then surgical intervention can be revisited.\n\nIn summary, the key points to consider are:\n\n1. Dyspnea is the primary symptom to address.\n2. Further evaluation is needed to determine the cause of the focus of air under the right hemidiaphragm.\n3. Conservative management should be the initial approach, with surgical intervention as a last resort.\n\nBy following this guidance, we can ensure that the patient receives appropriate and timely care while avoiding unnecessary interventions."}, {'role': 'Doctor_Intern', 'content': '```Diagnostic results: \nBased on the patient\'s medical record and radiology report, I can infer the following diagnosis:\n\n**Pulmonary Atelectasis**\nThe radiology report mentions "mild bibasilar atelectasis" which is a common finding on chest radiographs. Atelectasis is a condition where the airspaces in the lungs collapse or shrink, leading to reduced lung volume. The report also mentions that the lungs are otherwise clear, which suggests that the atelectasis is likely a benign finding and not related to any underlying lung disease.\n\nAdditionally, the report mentions "mild unfolding of the thoracic aorta" which could be a normal variant or an early sign of aortic dilatation, but it is not a significant finding in this case.\n\nThe patient\'s symptoms of dyspnea (shortness of breath) are likely related to the mild atelectasis, which is a common cause of respiratory symptoms.\n\nOther possible diagnoses that can be ruled out based on the report include:\n* Cardiopulmonary disease: The report mentions "No acute cardiopulmonary abnormality" which suggests that the patient does not have any significant cardiac or pulmonary disease.\n* Pneumothorax: The report mentions "Pleuralsurfaces are clear without effusion or pneumothorax" which rules out the possibility of a pneumothorax (collapsed lung).\n\nIt is important to note that a definitive diagnosis can only be made after a thorough physical examination and other diagnostic tests, such as a computed tomography (CT) scan or bronchoscopy.\n```\n\n```Current_service results: \nBased on the patient\'s medical record, I would recommend the \'MED\' service as the most suitable initial service for the patient. The patient\'s radiology report indicates that they have mild bibasilar atelectasis and mild unfolding of the thoracic aorta, which may be related to their symptoms of dyspnea. The report also states that there is no acute cardiopulmonary abnormality, but it is recommended to monitor the patient\'s condition further.\n\nThe \'MED\' service, which stands for Medical, is the most relevant service in this case as it deals with general medical conditions and symptoms such as dyspnea. A medical doctor would be able to assess the patient\'s condition, provide further guidance on their treatment plan, and monitor their progress.\n\nThe other services listed, such as \'ORTHO\' (Orthopedics), \'TSURG\' (Thoracic Surgery), and \'SURG\' (Surgery), may be more relevant if the patient had a surgical condition or injury, but based on the patient\'s current medical record, the \'MED\' service is the most suitable initial service.\n```'}, {'role': 'Doctor_Chief', 'content': 'Is it true?: Yes, the diagnosis of Pulmonary Atelectasis is correct based on the radiology report. The report mentions "mild bibasilar atelectasis" which is a common finding on chest radiographs. Atelectasis is a condition where the airspaces in the lungs collapse or shrink, leading to reduced lung volume. The report also mentions that the lungs are otherwise clear, which suggests that the atelectasis is likely a benign finding and not related to any underlying lung disease.\n\nGuidance: While the diagnosis of Pulmonary Atelectasis is correct, it\'s important to consider other potential causes of dyspnea. Atelectasis is a common cause of respiratory symptoms, but it\'s not the only one. It would be beneficial to conduct further tests such as a computed tomography (CT) scan or bronchoscopy to rule out other potential causes of dyspnea. Additionally, monitoring the patient\'s oxygen saturation levels and conducting a thorough physical examination would provide more insight into the patient\'s overall condition. \n\nRegarding the recommendation for the \'MED\' service, it is indeed the most suitable initial service for the patient. The \'MED\' service deals with general medical conditions and symptoms such as dyspnea, which aligns with the patient\'s current symptoms and diagnosis. A medical doctor would be able to assess the patient\'s condition, provide further guidance on their treatment plan, and monitor their progress. \n\nHowever, it\'s crucial to remember that a definitive diagnosis can only be made after a thorough physical examination and other diagnostic tests. Therefore, it\'s important to continue monitoring the patient\'s condition and adjust the treatment plan accordingly based on the results of these additional tests.'}, {'role': 'Doctor_Intern', 'content': "Diagnostic results: \n\nBased on the patient's medical record and radiology report, I can arrive at the following diagnosis:\n\nThe patient is a 45-year-old single white male who presents with dyspnea. The radiology report indicates mild bibasilar atelectasis, mild unfolding of the thoracic aorta, and a focus of air under the right hemidiaphragm, which is likely due to colonic interposition. The report also states that there is no acute cardiopulmonary abnormality.\n\nGiven these findings, I would diagnose the patient with Chronic Obstructive Pulmonary Disease (COPD), specifically chronic bronchitis. The mild bibasilar atelectasis and mild unfolding of the thoracic aorta are indicative of chronic lung disease, and the presence of a focus of air under the right hemidiaphragm is consistent with colonic interposition, which is a common complication of COPD.\n\nThe patient's past medical history and radiological findings do not suggest any acute cardiopulmonary abnormality, which supports the diagnosis of chronic bronchitis rather than acute respiratory failure.\n\nThe ICD-10 code for this diagnosis would be J44.9, which is the code for chronic bronchitis without exacerbation.\n\nCurrent_service results: \n\nBased on the patient's radiology report, which indicates mild bibasilar atelectasis and a focus of air seen under the right hemidiaphragm, likely representing colonic interposition, I would recommend the 'SURG' service as the most suitable option.\n\nThe patient's symptoms of dyspnea and the radiology findings suggest a possible respiratory issue, and the presence of colonic interposition may indicate a surgical issue. The 'SURG' service, which stands for Surgery, would be the most suitable option to further evaluate and manage the patient's condition."}]

In [14]:
aa[-1]["role"] +"'s final diagnostic:" + aa[-1]["content"]

"Doctor_Intern's final diagnosticDiagnostic results: \n\nBased on the patient's medical record and radiology report, I can arrive at the following diagnosis:\n\nThe patient is a 45-year-old single white male who presents with dyspnea. The radiology report indicates mild bibasilar atelectasis, mild unfolding of the thoracic aorta, and a focus of air under the right hemidiaphragm, which is likely due to colonic interposition. The report also states that there is no acute cardiopulmonary abnormality.\n\nGiven these findings, I would diagnose the patient with Chronic Obstructive Pulmonary Disease (COPD), specifically chronic bronchitis. The mild bibasilar atelectasis and mild unfolding of the thoracic aorta are indicative of chronic lung disease, and the presence of a focus of air under the right hemidiaphragm is consistent with colonic interposition, which is a common complication of COPD.\n\nThe patient's past medical history and radiological findings do not suggest any acute cardiopulmo